In [0]:
blob_account_name = "blob_account_name"
blob_container_name = "blob_container_name"
blob_relative_path = "excel/Tag Injection/"
blob_sas_token = dbutils.secrets.get("sas_token_name", "sastoken")
tableName = "taginjection"
mountLoc = "/mnt/taginjection/excel/"
excel_sheet="Azure Tag Injection.xlsx"

In [0]:
wasbs_path = f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/{blob_relative_path}"
spark.conf.set(f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",blob_sas_token)

In [0]:
from pyspark.sql.functions import lit
folder_path = f"{wasbs_path}"

spark.sql(f"DROP TABLE IF EXISTS {tableName}")

# Get the list of files and directories in the folder
items = dbutils.fs.ls(folder_path)

print(folder_path)
# Iterate over the items
for item in items:
    # Get the item path and name
    item_path = item.path
    item_name = item.name

    # dbutil subitem
    subItems = dbutils.fs.ls(item_path)

    for subItem in subItems:
        arraySplit = subItem.path.split("/")
        year = arraySplit[5]
        month = arraySplit[6]
        
        dbutils.fs.mount(
            source=f"{subItem.path}/{excel_sheet}",
            mount_point=f"{mountLoc}",
            extra_configs={
                f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net": blob_sas_token
            }
        )
        
        dfsub = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").option("dataAddress", "'Azure Tag Injection'!B1:K3000").load(mountLoc)

        dfsub = dfsub.withColumn("Year", lit(year))
        dfsub = dfsub.withColumn("Month", lit(month))
        
        dfsub.write.format("delta").mode("append").option("delta.columnMapping.mode", "name").saveAsTable(tableName)

        dbutils.fs.unmount(mountLoc)


In [0]:
dbutils.notebook.exit("OK")